<a href="https://colab.research.google.com/github/sanjayyanadi/Assignment_Sample/blob/sub/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize
import plotly.graph_objs as go
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
# Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive/')
df = pd.read_csv('/content/drive/MyDrive/complaints_processed.csv')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
df.head(30)

,Unnamed: 0,product,narrative
0,0,credit_card,purchase order day shipping amount receive pro...
1,1,credit_card,forwarded message date tue subject please inve...
2,2,retail_banking,forwarded message cc sent friday pdt subject f...
3,3,credit_reporting,payment history missing credit report speciali...
4,4,credit_reporting,payment history missing credit report made mis...
5,5,credit_reporting,payment history missing credit report made mis...
6,6,credit_reporting,va date complaint experian credit bureau invol...
7,7,credit_reporting,account reported abbreviated name full name se...
8,8,credit_reporting,account reported abbreviated name full name se...
9,9,credit_reporting,usdoexxxx account reported abbreviated name fu...


In [5]:
data = df.drop(['Unnamed: 0'], axis= 1)

In [6]:
data.groupby(['product']).count()

,narrative
product,
credit_card,15566
credit_reporting,91172
debt_collection,23148
mortgages_and_loans,18990
retail_banking,13535


In [7]:
def print_plot(index):
    example = data[data.index == index][['narrative', 'product']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Product:', example[1])

In [8]:
print_plot(1)
print_plot(11)
print_plot(21)
print_plot(31)

forwarded message date tue subject please investigate comenity bank retailer card scam sent hello name scammed comenity bank credit card provider company childrens place new york forever victoria secret original credit comenity bank lower limit began charge overage fee along late fee began pay close attention card find limit also changed well incurring overage late fee reached company comenity bank stated would change credit limit original limit reached told summit payment account corrected comenity bank credit card impacted credit score plummeted negative status im currently paying price due corruption affected detrimental way debt due company charging overage fee well late fee even initial credit limit fluctuating tremendously company charge major fee account willing correct account nervous said attorney reason im reaching im employee company ruining credit plz help name contact info thank
Product: credit_card
called request new york state covid relief plan day interest fee waived am

In [9]:
data = data.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
def clean_text(text):
    """
        text: a string
        return: modified initial string
    """
    text = str(text)
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    # remove stopwors from text
    return text
data['narrative'] = data['narrative'].apply(clean_text)
data['narrative'] = data['narrative'].str.replace('\d+', '')

<ipython-input-9-86dfc7624118>:18: FutureWarning:

The default value of regex will change from True to False in a future version.



In [10]:
print_plot(22)

converted lease vehicle three month lease end date sale dealer accepted deal including tax remaining payment existing lease deal work try convert lease vehicle sale office provide quote based current payoff amount car agreed bought vehicle problem sale manager dealer failed pay payment current lease instead paid two payment saying contract written manner checked contract paper nowhere mentioned converting lease vehicle new also financial document say current payoff amount much dealer agreed pay existing lease initially paid two payment ask benz financial service remove turn fee extra mileage called multiple remove well felt cheated deal providing proper information one payment need pay financial company also car sold used car knowledge
Product: mortgages_and_loans


In [11]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                      lower=True)
tokenizer.fit_on_texts(data['narrative'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 45960 unique tokens.


In [12]:
X = tokenizer.texts_to_sequences(data['narrative'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (162421, 250)


In [13]:
Y = pd.get_dummies(data['product']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (162421, 5)


In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.25, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(121815, 250) (121815, 5)
(40606, 250) (40606, 5)


In [15]:
## Model Architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(MAX_NB_WORDS, EMBEDDING_DIM,
                              input_length=X.shape[1]),
    tf.keras.layers.SpatialDropout1D(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,
                                                       return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    #tf.keras.layers.Dense(128, activation='softmax'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d (Spatial  (None, 250, 100)          0         
 Dropout1D)                                                      
                                                                 
 bidirectional (Bidirection  (None, 250, 512)          731136    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 256)               656384    
 onal)                                                           
                                                                 
 dense (Dense)               (None, 5)                 1285      
                                                        

In [16]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer='Nadam', metrics=["CategoricalAccuracy"])

In [17]:
## Make sure to use GPU before running this cell.
num_epochs = 5
batch_size = 128
## For early stopping to ensure it doesnt overfit
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)
history = model.fit(X_train, Y_train,
                    epochs=num_epochs, batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss',
                                             patience=3,
                                             min_delta=0.0001)])

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5562: UserWarning:

"`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?



857/857 [==============================] - 223s 243ms/step - loss: 0.7383 - categorical_accuracy: 0.7318 - val_loss: 0.8740 - val_categorical_accuracy: 0.6925
Epoch 2/5
857/857 [==============================] - 130s 152ms/step - loss: 0.5021 - categorical_accuracy: 0.8273 - val_loss: 0.4314 - val_categorical_accuracy: 0.8544
Epoch 3/5
857/857 [==============================] - 121s 141ms/step - loss: 0.3744 - categorical_accuracy: 0.8751 - val_loss: 0.4696 - val_categorical_accuracy: 0.8481
Epoch 4/5
857/857 [==============================] - 118s 138ms/step - loss: 0.3234 - categorical_accuracy: 0.8928 - val_loss: 0.3703 - val_categorical_accuracy: 0.8769
Epoch 5/5
857/857 [==============================] - 118s 138ms/step - loss: 0.2802 - categorical_accuracy: 0.9065 - val_loss: 0.3784 - val_categorical_accuracy: 0.8735


In [18]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

1269/1269 [==============================] - 28s 22ms/step - loss: 0.3862 - categorical_accuracy: 0.8703
Test set
  Loss: 0.386
  Accuracy: 0.870


In [ ]:
new_complaint = ['"I was charged twice for the same purchase on my credit card."']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = ['credit_card',
'credit_reporting',
'debt_collection',
'mortgages_and_loans',
'retail_banking']
print(pred, labels[np.argmax(pred)])

In [20]:
import pickle


In [21]:
# Assuming you have a trained model called 'model'
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)


In [22]:
from google.colab import files
files.download('model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>